**Run Notebook on Colab GCP VM Instance as CUDA is needed**

## Import Packages

In [ ]:
!nvidia-smi

Tue Nov 22 15:52:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8    14W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
import zipfile 
import gdown
import torch
from torch import positive, tensor
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import save_image, make_grid
from natsort import natsorted
from PIL import Image
import numpy as np
import pandas as pd
import random
from tqdm import tqdm
from torchvision import transforms
from google.colab import files

In [ ]:
# Zip File Creation Function
def back_up(folder):
  t = folder +'.zip'
  !zip -r $t $folder -qq
  files.download(t)
  print('zip file created')

## Download Image Files for CelebA

In [ ]:
## Fetch data from Google Drive
data_root = 'data'

# Path to download the dataset to - One Shot CelebA
img_download_path = f'data/train_celeba_one_shot.zip'
mapping_download_path = f'data/identity_CelebA.txt'

# Create required directories
if not os.path.exists(data_root):
  os.makedirs(data_root)

**Manually upload celeba zip file and mapping file to google colab**

In [ ]:
# Unzip the downloaded file 
with zipfile.ZipFile(img_download_path, 'r') as ziphandler:
  ziphandler.extractall(data_root)

  print('Dataset fully extracted.')

Dataset fully extracted.


In [ ]:
# Path to directory with all the images (train one shot, test, labels)
train_img_folder = 'data/train_celeba_one_shot'
mapping_file = 'data/identity_CelebA.txt'

In [ ]:
# Image File Names
img_file_names = os.listdir(train_img_folder)
img_file_names = natsorted(img_file_names)
img_file_names[:5]

['000001.jpg', '000002.jpg', '000003.jpg', '000004.jpg', '000005.jpg']

In [ ]:
# Image File Paths
img_file_paths = [os.path.join(train_img_folder, fname) for fname in img_file_names]
img_file_paths[:5]

['data/train_celeba_one_shot/000001.jpg',
 'data/train_celeba_one_shot/000002.jpg',
 'data/train_celeba_one_shot/000003.jpg',
 'data/train_celeba_one_shot/000004.jpg',
 'data/train_celeba_one_shot/000005.jpg']

In [ ]:
# File Nums to use later to save Augmented Images
f_num = [f.split('.jpg')[0] for f in img_file_names]
f_num[:5]

['000001', '000002', '000003', '000004', '000005']

In [ ]:
# Image File Label mapping
df = pd.read_csv(
            mapping_file, header=None, sep=" ", names=["file_name", "person_id"]
        )
df = df[df['file_name'].isin(img_file_names)] # only keep files in mapping file that are in the train_img_folder
df.head()

,file_name,person_id
0,000001.jpg,2880
1,000002.jpg,2937
2,000003.jpg,8692
3,000004.jpg,5805
4,000005.jpg,9295


In [ ]:
# make sure files are in order of file_name so indexing and label order is correct. Should be True.
df.equals(df.sort_values(by='file_name'))

True

In [ ]:
# Labels of Images in One Shot Train Folder
labels = df['person_id'].values.tolist()
labels[:5]

[2880, 2937, 8692, 5805, 9295]

In [ ]:
dataset_size = len(img_file_names)
print(f'Number of Images in Dataset: {dataset_size}')

Number of Images in Dataset: 10177


## Define Transforms for Images

In [ ]:
# Transformation of Images in CelebA
std_transform = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

resize_dims = (256, 256) # GAN output dimensions can always be changed in transforms for our facenet model

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


### Setup Cloning of HFGI Github Repo

In [ ]:
import os
os.chdir('/content')
CODE_DIR = 'HFGI'

In [ ]:
!git clone https://github.com/Tengfei-Wang/HFGI.git $CODE_DIR
!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!sudo unzip ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force

In [ ]:
os.chdir(f'./{CODE_DIR}')
from argparse import Namespace
import time
import os
import sys
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as transforms

sys.path.append(".")
sys.path.append("..")

from utils.common import tensor2im
from models.psp import pSp  # we use the pSp framework to load the e4e encoder.

%load_ext autoreload
%autoreload 2

## Download & Load Pretrained Models 
We provide pretrained models for face editing.

In [ ]:
#@title Download
def get_download_model_command(file_id, file_name):
    """ Get wget download command for downloading the desired model and save to directory pretrained_models. """
    current_directory = os.getcwd()
    save_path = os.path.join(os.path.dirname(current_directory), CODE_DIR, "checkpoint")
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    url = r"""wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id={FILE_ID}' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id={FILE_ID}" -O {SAVE_PATH}/{FILE_NAME} && rm -rf /tmp/cookies.txt""".format(FILE_ID=file_id, FILE_NAME=file_name, SAVE_PATH=save_path)
    return url    

path = {"id": "19y6pxOiJWB0NoG3fAZO9Eab66zkN9XIL", "name": "ckpt.pt"}
download_command = get_download_model_command(file_id=path["id"], file_name=path["name"]) 

!wget {download_command}

--2022-11-21 23:29:50--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2022-11-21 23:29:50--  https://docs.google.com/uc?export=download&confirm=t&id=19y6pxOiJWB0NoG3fAZO9Eab66zkN9XIL
Resolving docs.google.com (docs.google.com)... 142.250.148.100, 142.250.148.138, 142.250.148.139, ...
Connecting to docs.google.com (docs.google.com)|142.250.148.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0c-2k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/9apln8s4lgkav8f9v8a0ej7p9k23unbl/1669073325000/17817459031648051118/*/19y6pxOiJWB0NoG3fAZO9Eab66zkN9XIL?e=download&uuid=112758c2-642b-4ab3-ba31-29c82e963219 [following]
--2022-11-21 23:29:50--  https://doc-0c-2k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/9apln8s4lgkav8f9v8a0ej7p9k23unbl/1669073325000/17817459031648051118/*/19y6pxOiJWB0NoG3fAZO9Eab66zkN9XIL?e=download&u

In [ ]:
model_path = "checkpoint/ckpt.pt"
ckpt = torch.load(model_path, map_location='cpu')
opts = ckpt['opts']
opts['is_train'] = False
opts['checkpoint_path'] = model_path
opts= Namespace(**opts)
net = pSp(opts)
net.eval()
net.cuda()
print('Model successfully loaded!')

Loading basic encoder from checkpoint: checkpoint/ckpt.pt
Model successfully loaded!


## Image Alignment using Dlib

In [ ]:
import numpy as np
import PIL
import PIL.Image
import scipy
import scipy.ndimage
import dlib


def get_landmark(filepath, predictor):
    """get landmark with dlib
    :return: np.array shape=(68, 2)
    """
    detector = dlib.get_frontal_face_detector()

    img = dlib.load_rgb_image(filepath)
    dets = detector(img, 1)

    # dets is empty when dlib cannot detect a frontal face. Side profiles, etc
    if not dets:
      return None

    for k, d in enumerate(dets):
        shape = predictor(img, d)

    t = list(shape.parts())
    a = []
    for tt in t:
        a.append([tt.x, tt.y])
    lm = np.array(a)
    return lm


def align_face(filepath, predictor):
    """
    :param filepath: str
    :return: PIL Image
    """

    lm = get_landmark(filepath, predictor)

    # when dlib cannot detect frontal face, return None
    if lm is None:
      return None

    lm_chin = lm[0: 17]  # left-right
    lm_eyebrow_left = lm[17: 22]  # left-right
    lm_eyebrow_right = lm[22: 27]  # left-right
    lm_nose = lm[27: 31]  # top-down
    lm_nostrils = lm[31: 36]  # top-down
    lm_eye_left = lm[36: 42]  # left-clockwise
    lm_eye_right = lm[42: 48]  # left-clockwise
    lm_mouth_outer = lm[48: 60]  # left-clockwise
    lm_mouth_inner = lm[60: 68]  # left-clockwise

    # Calculate auxiliary vectors.
    eye_left = np.mean(lm_eye_left, axis=0)
    eye_right = np.mean(lm_eye_right, axis=0)
    eye_avg = (eye_left + eye_right) * 0.5
    eye_to_eye = eye_right - eye_left
    mouth_left = lm_mouth_outer[0]
    mouth_right = lm_mouth_outer[6]
    mouth_avg = (mouth_left + mouth_right) * 0.5
    eye_to_mouth = mouth_avg - eye_avg

    # Choose oriented crop rectangle.
    x = eye_to_eye - np.flipud(eye_to_mouth) * [-1, 1]
    x /= np.hypot(*x)
    x *= max(np.hypot(*eye_to_eye) * 2.0, np.hypot(*eye_to_mouth) * 1.8)
    y = np.flipud(x) * [-1, 1]
    c = eye_avg + eye_to_mouth * 0.1
    quad = np.stack([c - x - y, c - x + y, c + x + y, c + x - y])
    qsize = np.hypot(*x) * 2

    # read image
    img = PIL.Image.open(filepath)

    output_size = 256
    transform_size = 256
    enable_padding = True

    # Shrink.
    shrink = int(np.floor(qsize / output_size * 0.5))
    if shrink > 1:
        rsize = (int(np.rint(float(img.size[0]) / shrink)), int(np.rint(float(img.size[1]) / shrink)))
        img = img.resize(rsize, PIL.Image.ANTIALIAS)
        quad /= shrink
        qsize /= shrink

    # Crop.
    border = max(int(np.rint(qsize * 0.1)), 3)
    crop = (int(np.floor(min(quad[:, 0]))), int(np.floor(min(quad[:, 1]))), int(np.ceil(max(quad[:, 0]))),
            int(np.ceil(max(quad[:, 1]))))
    crop = (max(crop[0] - border, 0), max(crop[1] - border, 0), min(crop[2] + border, img.size[0]),
            min(crop[3] + border, img.size[1]))
    if crop[2] - crop[0] < img.size[0] or crop[3] - crop[1] < img.size[1]:
        img = img.crop(crop)
        quad -= crop[0:2]

    # Pad.
    pad = (int(np.floor(min(quad[:, 0]))), int(np.floor(min(quad[:, 1]))), int(np.ceil(max(quad[:, 0]))),
           int(np.ceil(max(quad[:, 1]))))
    pad = (max(-pad[0] + border, 0), max(-pad[1] + border, 0), max(pad[2] - img.size[0] + border, 0),
           max(pad[3] - img.size[1] + border, 0))
    if enable_padding and max(pad) > border - 4:
        pad = np.maximum(pad, int(np.rint(qsize * 0.3)))
        img = np.pad(np.float32(img), ((pad[1], pad[3]), (pad[0], pad[2]), (0, 0)), 'reflect')
        h, w, _ = img.shape
        y, x, _ = np.ogrid[:h, :w, :1]
        mask = np.maximum(1.0 - np.minimum(np.float32(x) / pad[0], np.float32(w - 1 - x) / pad[2]),
                          1.0 - np.minimum(np.float32(y) / pad[1], np.float32(h - 1 - y) / pad[3]))
        blur = qsize * 0.02
        img += (scipy.ndimage.gaussian_filter(img, [blur, blur, 0]) - img) * np.clip(mask * 3.0 + 1.0, 0.0, 1.0)
        img += (np.median(img, axis=(0, 1)) - img) * np.clip(mask, 0.0, 1.0)
        img = PIL.Image.fromarray(np.uint8(np.clip(np.rint(img), 0, 255)), 'RGB')
        quad += pad[:2]

    # Transform.
    img = img.transform((transform_size, transform_size), PIL.Image.QUAD, (quad + 0.5).flatten(), PIL.Image.BILINEAR)
    if output_size < transform_size:
        img = img.resize((output_size, output_size), PIL.Image.ANTIALIAS)

    # Return aligned image.
    return img

In [ ]:
if  'shape_predictor_68_face_landmarks.dat' not in os.listdir():
    !wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
    !bzip2 -dk shape_predictor_68_face_landmarks.dat.bz2

--2022-11-21 23:30:33--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  34.4MB/s    in 1.8s    

2022-11-21 23:30:35 (34.4 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]



In [ ]:
# change directory
os.chdir('/content')

In [ ]:
predictor = dlib.shape_predictor("HFGI/shape_predictor_68_face_landmarks.dat")

def run_alignment(image_path):
  aligned_image = align_face(filepath=image_path, predictor=predictor)
  if aligned_image is None: # dlib unable to detect frontal face
    return None
  return aligned_image

## Latent Code Functions for Inversions

In [ ]:
def display_alongside_source_image(result_image, source_image):
    res = np.concatenate([np.array(source_image.resize(resize_dims)),
                          np.array(result_image.resize(resize_dims))], axis=1)
    return Image.fromarray(res)

def get_latents(net, x, is_cars=False):
    codes = net.encoder(x)
    if net.opts.start_from_latent_avg:
        if codes.ndim == 2:
            codes = codes + net.latent_avg.repeat(codes.shape[0], 1, 1)[:, 0, :]
        else:
            codes = codes + net.latent_avg.repeat(codes.shape[0], 1, 1)
    if codes.shape[1] == 18 and is_cars:
        codes = codes[:, :16, :]
    return codes

### Running Alignment on Images in CelebA One Shot & Storing Latent Codes

In [ ]:
# Create Directory for Latent Codes
img_latent_codes_path = 'img_latent_codes'

if not os.path.exists(img_latent_codes_path):
  os.makedirs(img_latent_codes_path)

In [ ]:
# Create Directory for Distortion Maps 
img_distortions_path = 'img_distortion_maps'

if not os.path.exists(img_distortions_path):
  os.makedirs(img_distortions_path)

In [ ]:
# create file paths to store latent codes and distortion maps
l_paths = [os.path.join(img_latent_codes_path, name + '.pt') for name in f_num]
d_paths = [os.path.join(img_distortions_path, name +'.pt') for name in f_num]

In [ ]:
no_face = []
inference_times = [] # Store Inference Times Just to Understand Computation Time

for i, path in tqdm(enumerate(img_file_paths), total=len(img_file_paths)):  
  input_image = run_alignment(path) # should return a (256, 256) PIL Image

  if input_image is None: # dlib unable to detect frontal face
    no_face.append((img_file_names[i], i))
    input_image = Image.open(path).convert("RGB")

  img = std_transform(input_image) # perform transformation on image

  # Inversion Code of Images in Dataset
  with torch.no_grad():
    x = img.unsqueeze(0).cuda()

    # get latent codes
    tic = time.time()
    latent_codes = get_latents(net, x)
    torch.save(latent_codes[0], l_paths[i]) # (18, 512)
    
    # calculate the distortion map
    imgs, _ = net.decoder([latent_codes[0].unsqueeze(0).cuda()],None, input_is_latent=True, randomize_noise=False, return_latents=True)
    res = x -  torch.nn.functional.interpolate(torch.clamp(imgs, -1., 1.), size=(256,256) , mode='bilinear') # (1, 3, 256, 256)
    torch.save(res, d_paths[i]) # (1, 3, 256, 256)
      
    toc = time.time()
    inference_times.append(round(toc-tic, 4))


  0%|          | 0/10177 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
100%|██████████| 10177/10177 [1:18:08<00:00,  2.17it/s]


In [ ]:
print(f'Total Number of Images: {len(l_paths)}')
print(f'Number of Images Undetected by Dlib: {len(no_face)}')
print(f'Max Inference Time was : {max(inference_times)} seconds')
print(f'Median Inference Time was: {np.median(inference_times)} seconds')
print(f'Total Inference Time was: {sum(inference_times):4f} seconds')

Total Number of Images: 10177
Number of Images Undetected by Dlib: 343
Max Inference Time was : 0.4868 seconds
Median Inference Time was: 0.1186 seconds
Total Inference Time was: 1206.743600 seconds


## Load GAN Editings for Semantic Image Augmentation

In [ ]:
# change directory to access editings
os.chdir('HFGI')

from editings import latent_editor
editor = latent_editor.LatentEditor(net.decoder)

# change directory back to content to save files 
os.chdir('/content')

In [ ]:
# interface-GAN
interfacegan_directions = {
        'age': 'HFGI/editings/interfacegan_directions/age.pt',
        'smile': 'HFGI/editings/interfacegan_directions/smile.pt',
        'pose': 'HFGI/editings/interfacegan_directions/pose.pt' }

In [ ]:
# GANSpace
ganspace_pca = torch.load('HFGI/editings/ganspace_pca/ffhq_pca.pt') 
ganspace_directions = {
    'eyes':            (54,  7,  8,  20),
    'beard':           (58,  7,  9,  -20),
    'lip':             (34, 10, 11,  20) }            

## High-Fidelity Editing - Pose (INTERFACE GAN)
**(AUGMENTATION #1)**

In [ ]:
edit_direction = torch.load(interfacegan_directions['pose']).cuda()
edit_degree = 2

In [ ]:
# Create Directory to Save HGFI Pose Augmented Images
if not os.path.exists('celeba_train_hfgi_pose'):
  os.mkdir('celeba_train_hfgi_pose')

In [ ]:
pose_folder = 'celeba_train_hfgi_pose'

In [ ]:
# hfgi pose augmentation
pose_aug_fnames = []

for i in tqdm(range(dataset_size)):
  img_latent_code = torch.load(l_paths[i]) # tensor (18,512)
  img_edit, edit_latents = editor.apply_interfacegan(img_latent_code.unsqueeze(0).cuda(), edit_direction, factor=edit_degree)

  # align the distortion map
  img_edit = torch.nn.functional.interpolate(torch.clamp(img_edit, -1., 1.), size=(256,256) , mode='bilinear')
  img_distortion_map = torch.load(d_paths[i]) # tensor (1, 3, 256, 256)
  res_align  = net.grid_align(torch.cat((img_distortion_map, img_edit  ), 1))

  # fusion
  conditions = net.residue(res_align)
  result, _ = net.decoder([edit_latents],conditions, input_is_latent=True, randomize_noise=False, return_latents=True)

  result = torch.nn.functional.interpolate(result, size=(256,256) , mode='bilinear')

  #saving augmented images as jpg
  img_obj = tensor2im(result[0]) # (256, 256) Convert tensor (3, 256, 256) to Image (256, 256)
  aug_name = f_num[i] + '_' + 'pose' + '.jpg'
  pose_aug_fnames.append(aug_name)
  img_path = os.path.join(pose_folder, aug_name)
  img_obj.save(img_path)

  0%|          | 0/10177 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
100%|██████████| 10177/10177 [25:17<00:00,  6.71it/s]


In [ ]:
# create zip file to download augmented images
back_up(pose_folder)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

zip file created


In [ ]:
# create labels mapping file for data augmented images
zip_pose = list(zip(pose_aug_fnames, labels))
df_pose = pd.DataFrame(zip_pose, columns=["file_name", "person_id"])
df_pose.head()

,file_name,person_id
0,000001_pose.jpg,2880
1,000002_pose.jpg,2937
2,000003_pose.jpg,8692
3,000004_pose.jpg,5805
4,000005_pose.jpg,9295


In [ ]:
# Save mapping file
np.savetxt('labels_hfgi_pose_celeba.txt', df_pose, fmt = "%s")

## High-Fidelity Editing - Smile (INTERFACE GAN)
**(AUGMENTATION #2)**

In [ ]:
edit_direction = torch.load(interfacegan_directions['smile']).cuda()
edit_degree = 1.5

In [ ]:
# Create Directory to Save HGFI Smile Augmented Images
if not os.path.exists('celeba_train_hfgi_smile'):
  os.mkdir('celeba_train_hfgi_smile')

In [ ]:
smile_folder = 'celeba_train_hfgi_smile'

In [ ]:
# hfgi smile augmentation
smile_aug_fnames = []

for i in tqdm(range(dataset_size)):
  img_latent_code = torch.load(l_paths[i]) # tensor (18,512)
  img_edit, edit_latents = editor.apply_interfacegan(img_latent_code.unsqueeze(0).cuda(), edit_direction, factor=edit_degree)

  # align the distortion map
  img_edit = torch.nn.functional.interpolate(torch.clamp(img_edit, -1., 1.), size=(256,256) , mode='bilinear')
  img_distortion_map = torch.load(d_paths[i]) # tensor (1, 3, 256, 256)
  res_align  = net.grid_align(torch.cat((img_distortion_map, img_edit  ), 1))

  # fusion
  conditions = net.residue(res_align)
  result, _ = net.decoder([edit_latents],conditions, input_is_latent=True, randomize_noise=False, return_latents=True)

  result = torch.nn.functional.interpolate(result, size=(256,256) , mode='bilinear')

  #saving augmented images as jpg
  img_obj = tensor2im(result[0]) # (256, 256) Convert tensor (3, 256, 256) to Image (256, 256)
  aug_name = f_num[i] + '_' + 'smile' + '.jpg'
  smile_aug_fnames.append(aug_name)
  img_path = os.path.join(smile_folder, aug_name)
  img_obj.save(img_path)

  0%|          | 0/10177 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
100%|██████████| 10177/10177 [25:31<00:00,  6.65it/s]


In [ ]:
# create zip file to download augmented images
back_up(smile_folder)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

zip file created


In [ ]:
# create labels mapping file for data augmented images
zip_smile = list(zip(smile_aug_fnames, labels))
df_smile = pd.DataFrame(zip_smile, columns=["file_name", "person_id"])
df_smile.head()

,file_name,person_id
0,000001_smile.jpg,2880
1,000002_smile.jpg,2937
2,000003_smile.jpg,8692
3,000004_smile.jpg,5805
4,000005_smile.jpg,9295


In [ ]:
# Save mapping file
np.savetxt('labels_hfgi_smile_celeba.txt', df_smile, fmt = "%s")

## High-Fidelity Editing - Age (INTERFACE GAN)
**(AUGMENTATION #3)**

In [ ]:
edit_direction = torch.load(interfacegan_directions['age']).cuda()
edit_degree = 2.5

In [ ]:
# Create Directory to Save HGFI Age Augmented Images
if not os.path.exists('celeba_train_hfgi_age'):
  os.mkdir('celeba_train_hfgi_age')

In [ ]:
age_folder = 'celeba_train_hfgi_age'

In [ ]:
# hfgi age augmentation
age_aug_fnames = []

for i in tqdm(range(dataset_size)):
  img_latent_code = torch.load(l_paths[i]) # tensor (18,512)
  img_edit, edit_latents = editor.apply_interfacegan(img_latent_code.unsqueeze(0).cuda(), edit_direction, factor=edit_degree)

  # align the distortion map
  img_edit = torch.nn.functional.interpolate(torch.clamp(img_edit, -1., 1.), size=(256,256) , mode='bilinear')
  img_distortion_map = torch.load(d_paths[i]) # tensor (1, 3, 256, 256)
  res_align  = net.grid_align(torch.cat((img_distortion_map, img_edit  ), 1))

  # fusion
  conditions = net.residue(res_align)
  result, _ = net.decoder([edit_latents],conditions, input_is_latent=True, randomize_noise=False, return_latents=True)

  result = torch.nn.functional.interpolate(result, size=(256,256) , mode='bilinear')

  #saving augmented images as jpg
  img_obj = tensor2im(result[0]) # (256, 256) Convert tensor (3, 256, 256) to Image (256, 256)
  aug_name = f_num[i] + '_' + 'age' + '.jpg'
  age_aug_fnames.append(aug_name)
  img_path = os.path.join(age_folder, aug_name)
  img_obj.save(img_path)

  0%|          | 0/10177 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
100%|██████████| 10177/10177 [27:35<00:00,  6.15it/s]


In [ ]:
# create zip file to download augmented images
back_up(age_folder)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

zip file created


In [ ]:
# create labels mapping file for data augmented images
zip_age = list(zip(age_aug_fnames, labels))
df_age = pd.DataFrame(zip_age, columns=["file_name", "person_id"])
df_age.head()

,file_name,person_id
0,000001_age.jpg,2880
1,000002_age.jpg,2937
2,000003_age.jpg,8692
3,000004_age.jpg,5805
4,000005_age.jpg,9295


In [ ]:
# Save mapping file
np.savetxt('labels_hfgi_age_celeba.txt', df_age, fmt = "%s")

## High-Fidelity Editing - Eyes (GANSPACE)
**(AUGMENTATION #4)**

In [ ]:
edit_direction = ganspace_directions['eyes']

In [ ]:
# Create Directory to Save HGFI Eyes Augmented Images
if not os.path.exists('celeba_train_hfgi_eyes'):
  os.mkdir('celeba_train_hfgi_eyes')

In [ ]:
eyes_folder = 'celeba_train_hfgi_eyes'

In [ ]:
# hfgi eyes augmentation
eyes_aug_fnames = []

for i in tqdm(range(dataset_size)):
  img_latent_code = torch.load(l_paths[i]) # tensor (18,512)
  img_edit, edit_latents = editor.apply_ganspace(img_latent_code.unsqueeze(0).cuda(), ganspace_pca, [edit_direction])

  # align the distortion map
  img_edit = torch.nn.functional.interpolate(torch.clamp(img_edit, -1., 1.), size=(256,256) , mode='bilinear')
  img_distortion_map = torch.load(d_paths[i]) # tensor (1, 3, 256, 256)
  res_align  = net.grid_align(torch.cat((img_distortion_map, img_edit  ), 1))

  # fusion
  conditions = net.residue(res_align)
  result, _ = net.decoder([edit_latents],conditions, input_is_latent=True, randomize_noise=False, return_latents=True)

  result = torch.nn.functional.interpolate(result, size=(256,256) , mode='bilinear')

  #saving augmented images as jpg
  img_obj = tensor2im(result[0]) # (256, 256) Convert tensor (3, 256, 256) to Image (256, 256)
  aug_name = f_num[i] + '_' + 'eyes' + '.jpg'
  eyes_aug_fnames.append(aug_name)
  img_path = os.path.join(eyes_folder, aug_name)
  img_obj.save(img_path)

  0%|          | 0/10177 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
100%|██████████| 10177/10177 [25:33<00:00,  6.64it/s]


In [ ]:
# create zip file to download augmented images
back_up(eyes_folder)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

zip file created


In [ ]:
# create labels mapping file for data augmented images
zip_eyes = list(zip(eyes_aug_fnames, labels))
df_eyes = pd.DataFrame(zip_eyes, columns=["file_name", "person_id"])
df_eyes.head()

,file_name,person_id
0,000001_eyes.jpg,2880
1,000002_eyes.jpg,2937
2,000003_eyes.jpg,8692
3,000004_eyes.jpg,5805
4,000005_eyes.jpg,9295


In [ ]:
# Save mapping file
np.savetxt('labels_hfgi_eyes_celeba.txt', df_eyes, fmt = "%s")

## High-Fidelity Editing - Lip (GANSPACE)
**(AUGMENTATION #5)**

In [ ]:
edit_direction = ganspace_directions['lip']

In [ ]:
# Create Directory to Save HGFI Lip Augmented Images
if not os.path.exists('celeba_train_hfgi_lip'):
  os.mkdir('celeba_train_hfgi_lip')

In [ ]:
lip_folder = 'celeba_train_hfgi_lip'

In [ ]:
# hfgi lip augmentation
lip_aug_fnames = []

for i in tqdm(range(dataset_size)):
  img_latent_code = torch.load(l_paths[i]) # tensor (18,512)
  img_edit, edit_latents = editor.apply_ganspace(img_latent_code.unsqueeze(0).cuda(), ganspace_pca, [edit_direction])

  # align the distortion map
  img_edit = torch.nn.functional.interpolate(torch.clamp(img_edit, -1., 1.), size=(256,256) , mode='bilinear')
  img_distortion_map = torch.load(d_paths[i]) # tensor (1, 3, 256, 256)
  res_align  = net.grid_align(torch.cat((img_distortion_map, img_edit  ), 1))

  # fusion
  conditions = net.residue(res_align)
  result, _ = net.decoder([edit_latents],conditions, input_is_latent=True, randomize_noise=False, return_latents=True)

  result = torch.nn.functional.interpolate(result, size=(256,256) , mode='bilinear')

  #saving augmented images as jpg
  img_obj = tensor2im(result[0]) # (256, 256) Convert tensor (3, 256, 256) to Image (256, 256)
  aug_name = f_num[i] + '_' + 'lip' + '.jpg'
  lip_aug_fnames.append(aug_name)
  img_path = os.path.join(lip_folder, aug_name)
  img_obj.save(img_path)

  0%|          | 0/10177 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
100%|██████████| 10177/10177 [25:33<00:00,  6.64it/s]


In [ ]:
# create zip file to download augmented images
back_up(lip_folder)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

zip file created


In [ ]:
# create labels mapping file for data augmented images
zip_lip = list(zip(lip_aug_fnames, labels))
df_lip = pd.DataFrame(zip_lip, columns=["file_name", "person_id"])
df_lip.head()

,file_name,person_id
0,000001_lip.jpg,2880
1,000002_lip.jpg,2937
2,000003_lip.jpg,8692
3,000004_lip.jpg,5805
4,000005_lip.jpg,9295


In [ ]:
# Save mapping file
np.savetxt('labels_hfgi_lip_celeba.txt', df_lip, fmt = "%s")